# Tracing Without LangChain

LangSmith lets you instrument applications even if you don't want to depend on LangChain itself. The following is an example chat application using the raw openai SDK. First, install langsmith and the openai SDK

In [1]:
%pip install -U pip > /dev/null
%pip install -U langsmith > /dev/null
%pip install -U openai > /dev/null

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Next, configure the API Key in the environment to make sure traces are logged to your account.

In [ ]:
# %env LANGCHAIN_API_KEY=<your-api-key>

Next, define your chat application. Here we will use the `traceable` decorator.

In [2]:
import openai
import os
from datetime import datetime
from typing import List, Optional, Tuple
from langsmith.run_helpers import traceable

_SYSTEM_PROMPT = f"You are a helpful AI assistant. The current time is {datetime.now()}"

class Chat:
    
    def __init__(self, model: str = "gpt-3.5-turbo", api_key: Optional[str] = None, system_prompt: Optional[str] = _SYSTEM_PROMPT):
        """Initialize the Chat class with the specified model and set up memory.
        
        Args:
            model (str): The ID of the model to use for the conversation.
            api_key (str): The API key for authenticating with OpenAI.
        """
        openai.api_key = api_key or os.environ.get("OPENAI_API_KEY")
        
        self.model = model
        self.memory: List[Dict[str, str]] = []
        self._system_prompt = system_prompt
        self.reset_memory()
            
            
    def _insert_system_prompt(self) -> None:
        if self._system_prompt:
            self.memory.insert(0, {"role": "system", "content": self._system_prompt})

    @traceable(run_type="llm")
    def _call_openai(self, messages: List[List[dict]], model: str, temperature: float = 0.0):
        return openai.ChatCompletion.create(
            model=self.model,
            messages=messages[0],
            stream=True,
            temperature=temperature,
        )
    
    @traceable(run_type="chain")
    def __call__(self, text: str) -> str:
        """Make a call to the OpenAI API to get a response for the provided text.
        
        Args:
            text (str): The text input from the user.
        
        Returns:
            str: The model's response.
        """
        self.memory.append({"role": "user", "content":  text})
        response = self._call_openai([self.memory], self.model)
        total_resp = ""
        for chunk in response:
            delta = chunk.choices[0].delta
            if "content" in delta:
                print(delta.content, end="", flush=True)
                total_resp += delta.content
        self.memory.append({"role": "assistant", "content": total_resp})
        print()
        return total_resp
    

    def reset_memory(self):
        """Clear the chat memory."""
        self.memory = []
        self._insert_system_prompt()


The @traceable decorator is experimental and will likely see breaking changes.


In [3]:
chat = Chat()
chat("Hi")
chat("My name as Will")
result = chat("No assisting needed actually")

Hello! How can I assist you today?
Hello Will! How can I assist you today?
Alright, Will! If you have any questions or need assistance in the future, feel free to ask. Have a great day!


## Capturing the runs

The `traceable` decorator will inject the current `RunTree` object into the traced function if the argument is provided. You can use this to do things like fetch the run ID. Below, our `Chat2` app is an identical to the previous one but views the injected run_tree so we can fetch the latest run. The chat 

In [4]:
from langsmith import RunTree

class Chat2(Chat):
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.run_ids = []
    
    @traceable(run_type="chain")
    def __call__(self, text: str, run_tree: RunTree) -> str:
        self.run_ids.append(run_tree.id)
        return super().__call__(text)
        

In [5]:
chat = Chat2()

In [6]:
result = chat("Hi")
chat("My name is Will")
result = chat("No assisting needed actually")

Hello! How can I assist you today?
Hello, Will! How can I assist you today?
Alright, Will! If you have any questions or need assistance in the future, feel free to reach out. Have a great day!


In [7]:
latest_id = chat.run_ids[-1]

In [8]:
from langsmith import Client

client = Client()
shared_link = client.share_run(latest_id)

from IPython.display import IFrame
# Here's an example run:
IFrame(shared_link, width='1000px', height='520px', zoom=50)

In [9]:
shared_link

'https://dev.smith.langchain.com/public/922e9b80-19de-4c2c-bd50-ec9b02c2dc4f/r'

## Using the RunTree

The `traceable` decorator is lightweight but less flexible when it comes to defining the schema of the logs you want to save. Below, we will make an example of the chat application that logs runs using the RunTree object

In [10]:
import openai
import os
from datetime import datetime
from typing import List, Optional, Tuple
from langsmith import RunTree

_SYSTEM_PROMPT = f"You are a helpful AI assistant. The current time is {datetime.now()}"

class ChatWithRunTree:
    
    def __init__(self, model: str = "gpt-3.5-turbo", api_key: Optional[str] = None, system_prompt: Optional[str] = _SYSTEM_PROMPT):
        """Initialize the Chat class with the specified model and set up memory.
        
        Args:
            model (str): The ID of the model to use for the conversation.
            api_key (str): The API key for authenticating with OpenAI.
        """
        openai.api_key = api_key or os.environ.get("OPENAI_API_KEY")
        
        self.model = model
        self.memory: List[Dict[str, str]] = []
        self._system_prompt = system_prompt
        self.reset_memory()
        self.run_ids = []
            
            
    def _insert_system_prompt(self) -> None:
        if self._system_prompt:
            self.memory.insert(0, {"role": "system", "content": self._system_prompt})

    def _call_openai(self, messages: List[List[dict]], model: str, run_tree: RunTree = None, temperature: float = 0.0):
        child = run_tree.create_child(
            name="OpenAI.ChatCompletion",
            inputs={
                "messages": messages,
            },
            extra={
                "temperature": temperature,
                "model": model
            },
            run_type="llm",
        )
        child.post()
        try:
            resp = openai.ChatCompletion.create(
                model=self.model,
                messages=messages[0],
                stream=True,
                temperature=temperature,
            )
        except Exception as e:
            child.end(error=str(e))
            child.patch()
            raise
        child.end(outputs={"response": resp})
        child.patch()
        return resp
    
    def __call__(self, text: str) -> str:
        """Make a call to the OpenAI API to get a response for the provided text.
        
        Args:
            text (str): The text input from the user.
        
        Returns:
            str: The model's response.
        """
        tree = RunTree(name="Chat.__call__", run_type="chain", inputs={"text": text}, events=[])
        tree.post()
        try:
            self.memory.append({"role": "user", "content":  text})
            response = self._call_openai([self.memory], self.model, run_tree=tree)
            total_resp = ""
            for chunk in response:
                delta = chunk.choices[0].delta
                if "content" in delta:
                    # You can add any dictionaries as run events
                    tree.events.append(
                        {
                            "name": "on_token",
                            "content": delta.content,
                            "time": datetime.now()
                        }
                    )
                    print(delta.content, end="", flush=True)
                    total_resp += delta.content
            self.memory.append({"role": "assistant", "content": total_resp})
        except Exception as e:
            tree.end(error=str(e))
            tree.patch()
            raise e
        tree.end(outputs={"response": total_resp})
        tree.patch()
        self.run_ids.append(tree.id)
        print()
        return total_resp
    

    def reset_memory(self):
        """Clear the chat memory."""
        self.memory = []
        self._insert_system_prompt()


In [11]:
chat = ChatWithRunTree()

In [12]:
chat = ChatWithRunTree()
chat("Hi")
chat("My name as Will")
result = chat("No assisting needed actually")

Hello! How can I assist you today?
Hello Will! How can I assist you today?
Alright, Will! If you have any questions or need assistance in the future, feel free to ask. Have a great day!


In [13]:
latest_id = chat.run_ids[-1]
shared_link = client.share_run(latest_id)
IFrame(shared_link, width='1000px', height='520px', zoom=50)